*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data  = pd.read_csv(train_file_path, sep='\t', header = None)
train_data.columns = ["label", "message"]
train_data

In [ ]:
test_data  = pd.read_csv(test_file_path, sep='\t', header = None)
test_data.columns = ["label", "message"]
test_data

In [ ]:
y_train = train_data['label'].map({"spam": 1, "ham": 0})
y_train

In [ ]:
y_test = test_data['label'].map({"spam": 1, "ham": 0})
y_test

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

X_train = []
for _ in train_data['message'].values:
  X_train.append(_)

X_test = []
for _ in test_data['message'].values:
  X_test.append(_)

tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count
max_id
dataset_size

In [ ]:
encoded_train = []
for _ in X_train:
  encoded_train.append(tokenizer.texts_to_sequences([_])[0])

encoded_test = []
for _ in X_test:
  encoded_test.append(tokenizer.texts_to_sequences([_])[0])
encoded_train[:5]

In [ ]:
encoded_test[:5]

In [ ]:
from keras.preprocessing import sequence

MAXLEN = 1000

train = sequence.pad_sequences(encoded_train, MAXLEN)
test = sequence.pad_sequences(encoded_test, MAXLEN)
train
len(train[0])
len(train[1])

In [ ]:
test
len(test[0])
len(test[1])


In [ ]:
VOCAB_SIZE = max_id + 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ['acc'])

history = model.fit(train, y_train, epochs = 7, validation_split = 0.2)

In [ ]:
results = model.evaluate(test, y_test)
print(results)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = [pred_text]
  encoded_pred = []
  for _ in pred_text:
    encoded_pred.append(tokenizer.texts_to_sequences([_])[0])
  
  pred = sequence.pad_sequences(encoded_pred, MAXLEN)
  result = model.predict(pred)
  if result[0][0] < 0.5:
    label = "ham"
  else:
    label = "spam"
  prediction = [result[0][0], label]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
